In [33]:
import pandas as pd
import re

## Open data csv

In [34]:
data = pd.read_csv('../../data/data_2.csv')

In [35]:
data.head()

,Datetime,Text,Username,location,hashtags
0,2021-03-11 23:58:25+00:00,"Bustam, ketua PWI Papua Barat, mengajak para w...",ywfbruh,"Pennsylvania, USA",['OtsusPapuaTerbaik']
1,2021-03-11 23:56:03+00:00,Vaksinasi Diklaim Tingkatkan Jumlah Pengunjung...,Harian_Jogja,"Yogyakarta, Indonesia","['beritajogja', 'jogja', 'jogjaistimewa']"
2,2021-03-11 23:40:03+00:00,Wujudkan kekebalan komunal setelah vaksinasi d...,polres_pemalang,"Pemalang, Indonesia","['polriindonesia', 'polriuntukindonesia', 'kab..."
3,2021-03-11 23:38:51+00:00,Terima kasih kepada pemerintah atas perhatiann...,TomtomTheTITANS,bandung - jakarta,['VaksinUntukKita']
4,2021-03-11 23:36:44+00:00,"NU Jatim: Jangan Ikutin Hoaks, Vaksin Sinovac ...",BacaDiBaBe,Indonesia,"['MengenalVaksinAstraZeneca', 'PenerimaVaksinC..."


In [36]:
text = data['Text']
data_copy = data[['Datetime','Username','location']]

In [37]:
text

0       Bustam, ketua PWI Papua Barat, mengajak para w...
1       Vaksinasi Diklaim Tingkatkan Jumlah Pengunjung...
2       Wujudkan kekebalan komunal setelah vaksinasi d...
3       Terima kasih kepada pemerintah atas perhatiann...
4       NU Jatim: Jangan Ikutin Hoaks, Vaksin Sinovac ...
                              ...                        
7002    Lindungi Anak Dari Covid-19 Dengan Vaksinasi \...
7003    13.Kanwil Kemenkumham Sumsel: Vaksinasi tetap ...
7004    Ini Capaian Vaksin di SD Negeri 014 Kota Bangu...
7005    Vaksinasi tujuannya untuk melindungi anak dari...
7006    Dikasih info lagi maszzeh ... Yuk ikuti vaksin...
Name: Text, Length: 7007, dtype: object

In [38]:
data_copy.head()

,Datetime,Username,location
0,2021-03-11 23:58:25+00:00,ywfbruh,"Pennsylvania, USA"
1,2021-03-11 23:56:03+00:00,Harian_Jogja,"Yogyakarta, Indonesia"
2,2021-03-11 23:40:03+00:00,polres_pemalang,"Pemalang, Indonesia"
3,2021-03-11 23:38:51+00:00,TomtomTheTITANS,bandung - jakarta
4,2021-03-11 23:36:44+00:00,BacaDiBaBe,Indonesia


## Text processing

### Tokenizing word

In [39]:
def tokenizing(input):
  input = input.replace(".", "")
  input = input.replace(",", "")
  input = input.replace(":", "")
  input = input.replace("-", " ")
  input = input.replace("?", "")
  input = input.replace("!", "")
  input = input.replace("(", "")
  input = input.replace(")", "")
  input = input.replace("[", "")
  input = input.replace("]", "")
  input = input.replace("{", "")
  input = input.replace("}", "")
  input = input.replace("'", "")
  input = input.replace('"', "")
  input = input.replace("/", "")
  input = input.replace("\n", " ")
  return input

### Split word

In [40]:
def split_words(words):
  
  return words.split(' ')

### Stop word

In [41]:
# open stopword

def stop_word(word):

  f = open('../components/id.stopwords.02.01.2016.txt')
  s_word = f.read()

  if word not in s_word :
    return word
  

  f.close()

### Kamus bahasa indonesia

In [42]:
kamus_clean = []

with open('../components/kata-dasar-indonesia.txt') as f:
  kamus = f.read()
  kamus = kamus.split(' ')

  for w in kamus:
  
    rx = re.findall(r'[^\n]+$', w)

    kamus_clean.append(rx[0])

### word stemming

In [43]:
AKAR_KATA = []

def kamus_word(word):
    checked = False
    # kamus = map(clean_word, kamus)
    if word in kamus_clean:
      checked = True
      # AKAR_KATA.append(word)
      return checked, word 
    return checked, word
    

### Func for delete infleksional suffiks

In [56]:
def hapus_infleksional_suffiks(word):
  # akhiran -lah, -kah, -nya, -tah, -pun
  checked = False
  
  
  if word.endswith('lah') or word.endswith('kah') or word.endswith('nya') or word.endswith('tah') or word.endswith('pun'):
    word = word[0 : len(word) - 3]
    
    # check if word exist in kamus
    checked, word = kamus_word(word)
 
    return checked, word

  # akhiran -ku, -mu
  elif word.endswith('ku') or word.endswith('mu'):
    word = word[0 : len(word) - 2]

    # check if word exist in kamus
    checked, word = kamus_word(word)
    return checked, word
    
  return checked, word

### Func for derivation suffiks

In [45]:
def hapus_derivation_suffiks(word):
  checked = False
    # akhiran kan
  if word.endswith('kan') and len(word) > 4:
    word = word[0 : len(word) - 3]

    # check if word exist in kamus
    checked, word = kamus_word(word)
    return checked, word

  # akhiran i
  if word.endswith('i') and len(word) > 3:
    word = word[0 : len(word) - 1]

    # check if word exist in kamus
    checked, word = kamus_word(word)
    return checked, word

  # akhiran an
  if word.endswith('an') and len(word) > 4:
    word = word[0 : len(word) - 2]

    # check if word exist in kamus
    checked, word = kamus_word(word)
    return checked, word


  return checked, word

### Func for delete derivation prefiks

In [93]:
def hapus_derivation_prefiks(word):
  checked = False
  # awalan mempel-
  if (word.startswith('mempel')) and (len(word) > 6):
    sub_word = word[6:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:    
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word
      
    return checked, word
  
  # awalan memper-
  if (word.startswith('memper')) and (len(word) > 6):
    sub_word = word[6:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:  
      
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word

    return checked, word

  # awalan diper-, keber-, keter-
  if (word.startswith('diper') or word.startswith('keber') or word.startswith('keter')) and (len(word) > 5):
    sub_word = word[5:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:  

      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word
    return checked, word

  # awalan meng-, peng-
  if (word.startswith('meng') or word.startswith('peng')) and (len(word) > 4):
    sub_word = word[4:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)
      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)

      if checked==False:  
        # add 'k' in front word
        sub_word = 'k' + word

        # check word again in kamus
        checked, word = kamus_word(sub_word)

        if checked==False:  
          # check word in derivation suffiks
          checked, word_pref_suff_k = hapus_derivation_suffiks(word)

          # check word again in kamus after add k in front
          checked, word = kamus_word(word_pref_suff_k)
          return checked, word
        return checked, word
      return checked, word
    return checked, word

  # awalan meny-, peny-
  if (word.startswith('meny') or word.startswith('peny')) and (len(word) > 4):
    sub_word = word[4:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:  
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      if checked==False:  

        # add 's' in front word
        sub_word = 's' + word

        # check word again in kamus
        checked, word = kamus_word(sub_word)

        if checked==False:  
          # check word in derivation suffiks
          checked, word_pref_suff_s = hapus_derivation_suffiks(word)

          # check word again in kamus after add s in front
          checked, word = kamus_word(word_pref_suff_s)
        
          return checked, word
        return checked, word
      return checked, word

    return checked, word

  # awalan mel-, mer-, pel-, per-
  if (word.startswith('mel') or word.startswith('mer') or word.startswith('pel') or word.startswith('per')) and (len(word) > 3):
    sub_word = word[3:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)

    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)

      return checked, word
    return checked, word

  # awalan men-, pen-
  if (word.startswith('men') or word.startswith('pen')) and (len(word) > 3):
    sub_word = word[3:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)

    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)
      sub_word = 't' + word_pref_suff

      # check word again in kamus
      checked, word = kamus_word(sub_word)
      
      if checked==False: 
        # check word in derivation suffiks
        checked, word_pref_suff_t = hapus_derivation_suffiks(word)

        # check word again in kamus after add s in front
        checked, word = kamus_word(word_pref_suff_t)
        return checked, word
      return checked, word
    return checked, word

    
  # awalan mem-, pem-
  if (word.startswith('mem') or word.startswith('pem')) and (len(word) > 3):
    sub_word = word[3:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    
    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      if checked==False:  
        # add 's' in front word
        sub_word = 'p' + word
        
        # check word again in kamus
        checked, word = kamus_word(sub_word)
        
        
        if checked==False:     
          # check word in derivation suffiks
          checked, word_pref_suff_p = hapus_derivation_suffiks(word)

          # check word again in kamus after add s in front
          checked, word = kamus_word(word_pref_suff_p)
          return checked, word
        return checked, word
      return checked, word
    return checked, word
  
  # awalan bel-, ber-, tel-, ter-
  if (word.startswith('bel') or word.startswith('ber') or word.startswith('tel') or word.startswith('ter')) and (len(word) > 3):
    sub_word = word[3:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word
    return checked, word

  # tipe awalan ke 1 : di-, ke-, se-
  if (word.startswith('di') or word.startswith('ke') or word.startswith('se')) and (len(word) > 3):
    sub_word = word[2:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word
    return checked, word


  # tipe awalan ke 2 : be-, te-
  if (word.startswith('be') or word.startswith('te')) and (len(word) > 2):
    sub_word = word[2:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)
    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word

    return checked, word

  # tipe awalan ke 3 : me-, pe-
  if (word.startswith('me') or word.startswith('pe')) and (len(word) > 2):
    sub_word = word[2:]
    # check if word exist in kamus
    checked, word = kamus_word(sub_word)

    if checked==False:  
      # check word in derivation suffiks
      checked, word_pref_suff = hapus_derivation_suffiks(word)

      # check word again in kamus
      checked, word = kamus_word(word_pref_suff)
      return checked, word

    return checked, word
  return checked, word



## Stemming each text

In [91]:
list_text = []
for t in text.head():
    t = t.lower()
    t = tokenizing(t)
    t = split_words(t)

    list_text.append(t)
    print(t)

print('batas\n')
list_kamus = []
for word in list_text:
    words = []
    for w in word:
        # print(w)
        # cek kamus pertama
        checked,w = kamus_word(w)
        if (checked):
            words.append(w)
            continue

         # hapus derivation prefiks
        checked,w = hapus_derivation_prefiks(w)
        if checked:
            words.append(w)
            continue
            
        checked,w = hapus_infleksional_suffiks(w)
        if checked:
            words.append(w)
            continue
        
        # hapus derivation suffiks
        checked,w = hapus_derivation_suffiks(w)
        if checked:
            words.append(w)
            continue

        
       
    
    list_kamus.append(' '.join(words))
        


['bustam', 'ketua', 'pwi', 'papua', 'barat', 'mengajak', 'para', 'wartawan', 'mendukung', 'program', 'pemerintah', 'mencegah', 'penyebaran', 'covid', '19', 'salah', 'satunya', 'dengan', 'mendapat', 'vaksinasi', '#otsuspapuaterbaik', 'pon', 'xx', 'papua', 'siap', 'httpstcojprb6kboux']
['vaksinasi', 'diklaim', 'tingkatkan', 'jumlah', 'pengunjung', 'malioboro', 'httpstcogdt9otcswe', '#beritajogja', '#jogja', '#jogjaistimewa', 'httpstcofif4jxenim']
['wujudkan', 'kekebalan', 'komunal', 'setelah', 'vaksinasi', 'dengan', 'mematuhi', '5m', 'memakai', 'masker', 'mencuci', 'tangan', 'menjaga', 'jarak', 'menjauhi', 'kerumunan', 'dan', 'membatasi', 'mobilitas', '', '@ronny_tri2000', '@divhumas_polri', '@poldajateng_', '', '#polriindonesia', '#polriuntukindonesia', '#kabarpemalang', '#inipemalang', 'httpstco8ttrt4dqur']
['terima', 'kasih', 'kepada', 'pemerintah', 'atas', 'perhatiannya', 'sehingga', 'kalangan', 'difabel', 'mendapat', 'kesempatan', 'vaksinasi', 'dalam', 'waktu', 'yang', 'cepat', '', 

In [92]:
print(list_kamus)

['ketua barat ajak para wartawan dukung program perintah cegah sebar salah satu dengan dapat vaksinasi siap', 'vaksinasi klaim tingkat jumlah unjung', 'wujud kebal komunal telah vaksinasi dengan patuh masker cuci tangan jaga jarak dan batas mobilitas', 'terima kasih kepada perintah atas hati sehingga difabel dapat sempat vaksinasi dalam waktu yang cepat', 'jangan vaksin dan halal']


In [ ]:
text_stemming = pd.DataFrame(list_kamus, columns=['Text_after_stemming'])

In [ ]:
text_stemming

,Text_after_stemming
0,bagi kiat sukses vaksinasi
1,vaksinasi untuk umum kitar prioritas daerah pa...
2,vaksinasi untuk masyarakat umum ini akan prior...
3,vaksinasi untuk umum kitar prioritas daerah pa...
4,vaksinasi untuk umum kitar prioritas daerah pa...


### Concat data with Text_after_stemming

In [ ]:
data_copy = data

In [ ]:
data_with_text_stemming = data_copy.assign(Text_stemming=text_stemming)

In [49]:
data_with_text_stemming.head(10)

,Datetime,Text,Username,location,Text_stemming
0,2021-02-11 23:48:45+00:00,Sekda Aceh Berbagi Kiat Sukses Vaksinasi Covid...,Farder11,NaN,bagi kiat sukses vaksinasi
1,2021-02-11 23:47:54+00:00,Moeldoko: Vaksinasi Covid-19 untuk Umum Sekita...,zulitaufik,Jakarta,vaksinasi untuk umum kitar prioritas daerah pa...
2,2021-02-11 23:39:33+00:00,Vaksinasi untuk masyarakat umum ini akan mempr...,kompascom,Jakarta,vaksinasi untuk masyarakat umum ini akan prior...
3,2021-02-11 23:30:02+00:00,Moeldoko: Vaksinasi Covid-19 untuk Umum Sekita...,aldotjahjadi8,ihsgjournal@gmail.com,vaksinasi untuk umum kitar prioritas daerah pa...
4,2021-02-11 23:23:31+00:00,Moeldoko: Vaksinasi Covid-19 untuk Umum Sekita...,cosmofemalefm,"Kota Manado, Sulawesi Utara",vaksinasi untuk umum kitar prioritas daerah pa...
5,2021-02-11 23:02:20+00:00,#BuletinBernama: Vaksinasi COVID-19 bermula Ma...,BernamaTV,Kuala Lumpur,NaN
6,2021-02-11 23:02:15+00:00,Vaksinasi bagi para lansia untuk memperkuat im...,hasimramadan02,NaN,NaN
7,2021-02-11 23:00:25+00:00,Dengarkan bicara vaksin COVID-19 &amp; kepenti...,JKJAVMY,Malaysia,NaN
8,2021-02-11 22:41:38+00:00,"Sebagai Saran buat Bapak Menteri Desa PDTT , d...",IsidorusArwan3,NaN,NaN
9,2021-02-11 22:20:39+00:00,"Kapolsek Baturetno AKP DIYATNO, SH beserta Dan...",kota_sate,"Baturetno, Wonogiri, Indonesia",NaN


### Convert Data to csv

In [ ]:
csv_data = data_with_text_stemming.to_csv('../../data/data_2_stemming.csv', index=False)

## Lexicon

In [ ]:
lexicon_neg = pd.read_csv('../components/InSet-master/negative.tsv', sep="\t")
lexicon_pos = pd.read_csv('../components/InSet-master/positive.tsv', sep="\t")

In [ ]:
lexicon_neg.head()

,word,weight
0,putus tali gantung,-2
1,gelebah,-2
2,gobar hati,-2
3,tersentuh (perasaan),-1
4,isak,-5


In [ ]:
lexicon_pos.head()

,word,weight
0,hai,3
1,merekam,2
2,ekstensif,3
3,paripurna,1
4,detail,2


In [ ]:
test_text = text_stemming['Text_after_stemming']

In [ ]:
split = split_words(test_text[0])


In [ ]:
split

['bagi', 'kiat', 'sukses', 'vaksinasi']

In [ ]:
neg = 0
for w in split:
    for ind in lexicon_neg.index:
        # print(lexicon_neg['word'][ind], lexicon_neg['weight'][ind])

        if lexicon_neg['word'][ind] == w :
            neg += lexicon_neg['weight'][ind]

In [ ]:
pos = 0
for w in split:
    for ind in lexicon_pos.index:
        # print(lexicon_neg['word'][ind], lexicon_neg['weight'][ind])

        if lexicon_pos['word'][ind] == w :
            pos += lexicon_pos['weight'][ind]

In [ ]:
neg

-4

In [ ]:
pos

8

In [ ]:
pos = 0
neg = 0

for w in split:
    

SyntaxError: unexpected EOF while parsing (4185179895.py, line 5)